# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,5.46,81,61,1.37,RU,1726298231
1,1,yuzhno-kurilsk,44.0306,145.8556,15.37,68,100,4.69,RU,1726298232
2,2,port-aux-francais,-49.3500,70.2167,2.71,80,58,10.53,TF,1726298233
3,3,grytviken,-54.2811,-36.5092,-1.03,88,22,6.23,GS,1726298234
4,4,waitangi,-43.9535,-176.5597,9.21,66,9,3.14,NZ,1726298236


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points(x='Lng',y='Lat',size='Humidity',geo=True,tiles='OSM',width=800,height=600)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df=city_data_df[(city_data_df['Max Temp']<27)&(city_data_df['Max Temp']>21)&(city_data_df['Wind Speed']<4.5)&(city_data_df['Cloudiness']==0)]

# Drop any rows with null values
cleaned_ideal_weather_df=ideal_weather_df.dropna()
# Display sample data
cleaned_ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,tamanrasset,22.7850,5.5228,24.95,53,0,1.03,DZ,1726298237
73,73,mugla,37.2303,28.3557,25.26,38,0,1.92,TR,1726298318
138,138,margaret river,-33.9500,115.0667,22.29,45,0,2.22,AU,1726298395
147,147,tsiombe,-25.3000,45.4833,25.87,44,0,3.21,MG,1726298406
150,150,ridgecrest,35.6225,-117.6709,24.92,27,0,3.60,US,1726298237
185,185,laguna,38.4210,-121.4238,21.08,59,0,2.06,US,1726298452
198,198,remire-montjoly,4.9167,-52.2667,25.02,94,0,1.03,GF,1726298466
206,206,childress,34.4264,-100.2040,25.14,65,0,3.60,US,1726298477
207,207,ikalamavony,-21.1500,46.5833,24.57,38,0,3.87,MG,1726298478
229,229,stephenville,32.2207,-98.2023,24.52,79,0,1.54,US,1726298502


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=city_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bilibino,RU,68.0546,166.4372,81,
1,yuzhno-kurilsk,RU,44.0306,145.8556,68,
2,port-aux-francais,TF,-49.3500,70.2167,80,
3,grytviken,GS,-54.2811,-36.5092,88,
4,waitangi,NZ,-43.9535,-176.5597,66,
...,...,...,...,...,...,...
537,trail,CA,49.0998,-117.7022,67,
538,rabaul,PG,-4.1967,152.1721,86,
539,xinyuan,CN,37.3000,99.0000,94,
540,santa ana,US,33.7456,-117.8678,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,"limit":1,"filter":"","bias":""
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bilibino - nearest hotel: No hotel found
yuzhno-kurilsk - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
cabedelo - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
marka - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
wuyishan - nearest hotel: No hotel found
pumiao - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
kotel'nikovo - nearest hotel: No hotel found
sitka - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
qaqorto

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bilibino,RU,68.0546,166.4372,81,No hotel found
1,yuzhno-kurilsk,RU,44.0306,145.8556,68,No hotel found
2,port-aux-francais,TF,-49.3500,70.2167,80,No hotel found
3,grytviken,GS,-54.2811,-36.5092,88,No hotel found
4,waitangi,NZ,-43.9535,-176.5597,66,No hotel found
...,...,...,...,...,...,...
537,trail,CA,49.0998,-117.7022,67,No hotel found
538,rabaul,PG,-4.1967,152.1721,86,No hotel found
539,xinyuan,CN,37.3000,99.0000,94,No hotel found
540,santa ana,US,33.7456,-117.8678,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot=hotel_df.hvplot.points(x='Lng',y='Lat',size='Humidity',geo=True,tiles='OSM',width=800,height=600,hover_cols=['Hotel Name'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)